In [1]:
import requests
import boto3
from datetime import datetime

In [2]:
# AWS Configuration
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('yelp-restaurants')

# OpenSearch Configuration
opensearch_endpoint = ""
opensearch_headers = {"Content-Type": "application/json"}
auth = ("", "")  

# Yelp API Configuration
API_KEY = ""
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
SEARCH_LIMIT = 50

In [3]:
from datetime import datetime, timezone

def construct_url_params(url, params):
    # Construct URL parameters
    url += '?' if '?' not in url else '&'
    url += '&'.join([f"{key}={value}" for key, value in params.items()])
    return url

def business_search(cuisine, location="Manhattan"):
    # Search for businesses using the Yelp API
    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        "term": cuisine,
        "location": location,
        "limit": SEARCH_LIMIT
    }
    search_url = construct_url_params(url, params)
    response = requests.get(search_url, headers=HEADERS)

    if response.status_code == 200:
        return response.json()['businesses']
    else:
        print(f"Error: {response.json()}")
        return []

from decimal import Decimal

def store_in_dynamodb(restaurant):
    # Store restaurant data in DynamoDB
    try:
        table.put_item(
            Item={
                "business_id": restaurant["id"],
                "name": restaurant["name"],
                "address": ", ".join(restaurant["location"]["display_address"]),
                "coordinates": {
                    "latitude": Decimal(str(restaurant["coordinates"]["latitude"])),
                    "longitude": Decimal(str(restaurant["coordinates"]["longitude"]))
                },
                "review_count": restaurant["review_count"],
                "rating": Decimal(str(restaurant["rating"])),
                "zip_code": restaurant["location"]["zip_code"],
                "insertedAtTimestamp": datetime.now(timezone.utc).isoformat(),
            }
        )
        print(f"Stored in DynamoDB: {restaurant['name']}")
    except Exception as e:
        print(f"Error storing {restaurant['name']}: {e}")


def index_in_opensearch(restaurant):
    # Index restaurant data in OpenSearch
    try:
        restaurant_data = {
            "RestaurantID": restaurant["id"],
            "Cuisine": restaurant["categories"][0]["title"]
        }
        
        response = requests.put(
            f"{opensearch_endpoint}/restaurants/_doc/{restaurant['id']}",
            json=restaurant_data,
            headers=opensearch_headers,
            auth=auth
        )
        if response.status_code in [200, 201]:
            print(f"Indexed in OpenSearch: {restaurant['name']}")
        else:
            print(f"Error indexing: {response.text}")
    except Exception as e:
        print(f"Error: {e}")


In [13]:
def main():
    cuisines = ['Italian', 'Indian', 'French', 'Chinese', 'Mexican', 'Thai', 'Japanese']

    for cuisine in cuisines:
        print(f"Fetching {cuisine} restaurants...")
        restaurants = business_search(cuisine)

        for restaurant in restaurants:
            store_in_dynamodb(restaurant)
            index_in_opensearch(restaurant)

if __name__ == "__main__":
    main()

Fetching Italian restaurants...
Stored in DynamoDB: Olio e Più
Indexed in OpenSearch: Olio e Più
Stored in DynamoDB: Da Andrea - Chelsea
Indexed in OpenSearch: Da Andrea - Chelsea
Stored in DynamoDB: Piccola Cucina Uptown
Indexed in OpenSearch: Piccola Cucina Uptown
Stored in DynamoDB: Da Andrea - Greenwich Village
Indexed in OpenSearch: Da Andrea - Greenwich Village
Stored in DynamoDB: Da Raffaele
Indexed in OpenSearch: Da Raffaele
Stored in DynamoDB: Trattoria Trecolori
Indexed in OpenSearch: Trattoria Trecolori
Stored in DynamoDB: Rubirosa
Indexed in OpenSearch: Rubirosa
Stored in DynamoDB: La Pecora Bianca Bryant Park
Indexed in OpenSearch: La Pecora Bianca Bryant Park
Stored in DynamoDB: Carlotto
Indexed in OpenSearch: Carlotto
Stored in DynamoDB: Scarpetta
Indexed in OpenSearch: Scarpetta
Stored in DynamoDB: Torrisi
Indexed in OpenSearch: Torrisi
Stored in DynamoDB: L'Artusi
Indexed in OpenSearch: L'Artusi
Stored in DynamoDB: Carbone
Indexed in OpenSearch: Carbone
Stored in Dynam